In [18]:
from numba import cuda
cuda

<module 'numba.cuda' from 'C:\\Users\\ICT01_14\\.conda\\envs\\tf_test\\lib\\site-packages\\numba\\cuda\\__init__.py'>

In [ ]:
pip install --pre --upgrade mxnet-cu100
#pip install --pre --upgrade mxnet-cu90
pip install -U https://github.com/dmlc/gluon-nlp/archive/master.zip

In [1]:
import pandas as pd
import numpy as np
from mxnet.gluon import nn, rnn
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import time
import itertools
import random

### 버트 로딩 

In [2]:
ctx = mx.gpu(0)

In [3]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='wiki_multilingual_cased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
#print(bert_base)

In [5]:
ds = gluon.data.SimpleDataset([['나 보기가 역겨워', '김소월']])

tok = nlp.data.BERTTokenizer(vocab=vocabulary, lower=False)

trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=10)

list(ds.transform(trans))

[(array([    2,  8982,  9356, 47869,  9566,     3,  8935, 22333, 38851,
             3]), array(10), array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1]))]

In [7]:
#데이터는 https://github.com/e9t/nsmc 다운받는다. 
dataset_train = nlp.data.TSVDataset("ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [8]:
class BERTDataset(mx.gluon.data.Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        sent_dataset = gluon.data.SimpleDataset([[
            i[sent_idx],
        ] for i in dataset])
        self.sentences = sent_dataset.transform(transform)
        self.labels = gluon.data.SimpleDataset(
            [np.array(np.int32(i[label_idx])) for i in dataset])

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [9]:
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=False)
max_len = 64

In [10]:
data_train = BERTDataset(dataset_train, 0, 1, bert_tokenizer, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, bert_tokenizer, max_len, True, False)

In [11]:
class BERTClassifier(nn.Block):
    def __init__(self,
                 bert,
                 num_classes=2,
                 dropout=None,
                 prefix=None,
                 params=None):
        super(BERTClassifier, self).__init__(prefix=prefix, params=params)
        self.bert = bert
        with self.name_scope():
            self.classifier = nn.HybridSequential(prefix=prefix)
            if dropout:
                self.classifier.add(nn.Dropout(rate=dropout))
            self.classifier.add(nn.Dense(units=num_classes))

    def forward(self, inputs, token_types, valid_length=None):
        _, pooler = self.bert(inputs, token_types, valid_length)
        return self.classifier(pooler)
                                           

In [12]:
model = BERTClassifier(bert_base, num_classes=2, dropout=0.3)
# 분류 레이어만 초기화 한다. 
model.classifier.initialize(ctx=ctx)
model.hybridize()

# softmax cross entropy loss for classification
loss_function = gluon.loss.SoftmaxCELoss()

metric = mx.metric.Accuracy()

In [16]:
batch_size = 128
lr = 5e-5

train_dataloader = mx.gluon.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = mx.gluon.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

BrokenPipeError: [Errno 32] Broken pipe

In [12]:
trainer = gluon.Trainer(model.collect_params(), 'bertadam',
                        {'learning_rate': lr, 'epsilon': 1e-9, 'wd':0.01})

log_interval = 4
num_epochs = 4

In [13]:
# LayerNorm과 Bias에는 Weight Decay를 적용하지 않는다. 
for _, v in model.collect_params('.*beta|.*gamma|.*bias').items():
    v.wd_mult = 0.0
params = [
    p for p in model.collect_params().values() if p.grad_req != 'null'
]


In [14]:
def evaluate_accuracy(model, data_iter, ctx=ctx):
    acc = mx.metric.Accuracy()
    i = 0
    for i, (t,v,s, label) in enumerate(data_iter):
        token_ids = t.as_in_context(ctx)
        valid_length = v.as_in_context(ctx)
        segment_ids = s.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = model(token_ids, segment_ids, valid_length.astype('float32'))
        acc.update(preds=output, labels=label)
        if i > 1000:
            break
        i += 1
    return(acc.get()[1])

In [15]:
#learning rate warmup을 위한 준비 
step_size = batch_size 
num_train_examples = len(data_train)
num_train_steps = int(num_train_examples / step_size * num_epochs)
warmup_ratio = 0.1
num_warmup_steps = int(num_train_steps * warmup_ratio)
step_num = 0

In [16]:
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        step_num += 1
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            offset = (step_num - num_warmup_steps) * lr / (
                num_train_steps - num_warmup_steps)
            new_lr = lr - offset
        trainer.set_learning_rate(new_lr)
        with mx.autograd.record():
            # load data to GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # forward computation
            out = model(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # backward computation
        ls.backward()
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(token_ids.shape[0])

        step_loss += ls.asscalar()
        metric.update([label], [out])
        if (batch_id + 1) % (50) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.10f}, acc={:.3f}'
                         .format(epoch_id + 1, batch_id + 1, len(train_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0
    test_acc = evaluate_accuracy(model, test_dataloader, ctx)
    print('Test Acc : {}'.format(test_acc))

[Epoch 1 Batch 50/2344] loss=8.4847, lr=0.0000026681, acc=0.556
[Epoch 1 Batch 100/2344] loss=7.6343, lr=0.0000053362, acc=0.612
[Epoch 1 Batch 150/2344] loss=6.7952, lr=0.0000080043, acc=0.648
[Epoch 1 Batch 200/2344] loss=6.1983, lr=0.0000106724, acc=0.676
[Epoch 1 Batch 250/2344] loss=6.3750, lr=0.0000133404, acc=0.691
[Epoch 1 Batch 300/2344] loss=5.9387, lr=0.0000160085, acc=0.705
[Epoch 1 Batch 350/2344] loss=5.7424, lr=0.0000186766, acc=0.717
[Epoch 1 Batch 400/2344] loss=5.7524, lr=0.0000213447, acc=0.726
[Epoch 1 Batch 450/2344] loss=5.2769, lr=0.0000240128, acc=0.735
[Epoch 1 Batch 500/2344] loss=5.5889, lr=0.0000266809, acc=0.740
[Epoch 1 Batch 550/2344] loss=5.3694, lr=0.0000293490, acc=0.745
[Epoch 1 Batch 600/2344] loss=5.1612, lr=0.0000320171, acc=0.751
[Epoch 1 Batch 650/2344] loss=5.1401, lr=0.0000346852, acc=0.756
[Epoch 1 Batch 700/2344] loss=5.1864, lr=0.0000373533, acc=0.760
[Epoch 1 Batch 750/2344] loss=5.3412, lr=0.0000400213, acc=0.762
[Epoch 1 Batch 800/2344] l